In [ ]:
import os
import copy
import shutil
import pandas as pd

In [ ]:
# set path
# first copy original files into fmt_path
raw_path = "D:\\EGI_DATA\\EGI_RAW"
fmt_path = "D:\\EGI_DATA\\EGI_FMT"
edf_path = "D:\\EGI_DATA\\EGI_EDF"
bids_root_path = "D:\\EGI_DATA\\EGI_BIDS"
beh_path = "beh"
rec_path = "rec"
eeg_path = "eeg"
# set task name
wordtask = "wordchoice"
imagetask = "imagechoice"
foodtask = "foodchoice"
wmtask = "wordimagematch"
rest = "rest"

# 1. format raw data

## delete screen recording

In [ ]:
# for sub in os.listdir(fmt_path):
#     for direc in os.listdir(os.path.join(fmt_path, sub)):
#         if os.path.exists(os.path.join(fmt_path, sub, "Screencapture")):
#             shutil.rmtree(os.path.join(fmt_path, sub, "Screencapture"))

In [ ]:
if os.path.exists(fmt_path):
    shutil.rmtree(fmt_path)
else:
    os.mkdir(fmt_path)
task_set = set(["foodEnd", "foodChoice", "foodHealthy", "imageChoice", "wordChoice", "wordImageMatch", "foodTaste"])
task_dict = {"F": "foodchoice", "FC": "foodchoice", "I": "imagechoice", "IC": "imagechoice", "W": "wordchoice", "WC": "wordchoice", "R": "rest", "RS": "rest", "WM": "wordimagematch", "WM": "wordimagematch"}
total = 0
for sub in os.listdir(raw_path):
    # under "sub-xxx" directory
    if not os.path.exists(os.path.join(fmt_path, sub, eeg_path)):
        os.makedirs(os.path.join(fmt_path, sub, eeg_path))
    if not os.path.exists(os.path.join(fmt_path, sub, beh_path)):
        os.makedirs(os.path.join(fmt_path, sub, beh_path))
    for directory in os.listdir(os.path.join(raw_path, sub)):
        # eeg data
        if "EEG DATA" in directory or "EEG_DATA" in directory:
            # sub-001 to sub-015
            cur_eeg_path = os.path.join(raw_path, sub, directory)
            if sub <= "sub-015":
                cur_eeg_path = os.path.join(cur_eeg_path, "ori_mff")
            for mff in os.listdir(cur_eeg_path):
                if "fil.mff" in mff or "fil.edf" in mff:
                    continue
                for key, value in task_dict.items():
                    if key in mff.upper().replace('-', ' ').replace('_', ' ').split():
                        print(os.path.join(cur_eeg_path, mff))
                        shutil.copytree(os.path.join(cur_eeg_path, mff), os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.mff".format(sub, value)))
                        break
        # behavioral data
        elif "Beha" in directory:
            for csv in os.listdir(os.path.join(raw_path, sub, directory)):
                cross = set(csv.split("_")) & task_set
                if len(cross) != 0:
                    total += 1
                    # print(os.path.join(fmt_path, sub, beh_path, csv))
                    shutil.copy(os.path.join(raw_path, sub, directory, csv), os.path.join(fmt_path, sub, beh_path, "{}_task-{}_beh.csv".format(sub, list(cross)[0]).lower()))

## delete behavioral data and wordimagematch eeg data

In [ ]:
for sub in os.listdir(fmt_path):
    shutil.rmtree(os.path.join(fmt_path, sub, beh_path), ignore_errors=True)
    shutil.rmtree(os.path.join(fmt_path, sub, eeg_path, "{}_task-{}_eeg.mff".format(sub, wmtask)), ignore_errors=True)

# 2. convert .mff file to .edf file

## convert mff to edf

In [ ]:
from save_edf import write_mne_edf

In [ ]:
if os.path.exists(edf_path):
    shutil.rmtree(edf_path)
else:
    os.mkdir(edf_path)

In [ ]:
# place edf files in edf_path
for sub in os.listdir(fmt_path):
    # create sub/eeg path if not exist
    if not os.path.exists(os.path.join(edf_path, sub)):
        os.makedirs(os.path.join(edf_path, sub, eeg_path))
    for mff in os.listdir(os.path.join(fmt_path, sub, eeg_path)):
        if 
        edf_file_path = os.path.join(edf_path, sub, eeg_path, "{}edf".format(mff[:-3]))
        if not os.path.exists(edf_file_path):
            write_mne_edf(os.path.join(fmt_path, sub, eeg_path, mff), edf_file_path, overwrite=True)

# 3. BIDS

## convert to bids

In [ ]:
import mne
from mne_bids import write_raw_bids, BIDSPath, print_dir_tree

In [ ]:
if os.path.exists(bids_root_path):
    shutil.rmtree(bids_root_path)
else:
    os.makedirs(bids_root_path)

In [ ]:
# TODO
for sub in os.listdir(edf_path):
    # create sub/eeg path if not exist
    # if not os.path.exists(os.path.join(edf_path, sub)):
    #     os.makedirs(os.path.join(edf_path, sub, eeg_path))
    for edf_file in os.listdir(os.path.join(edf_path, sub, eeg_path)):
        subject_id = edf_file.split("_")[0].split("-")[1]
        task = edf_file.split("_")[1].split("-")[1]
        bids_path = BIDSPath(subject=subject_id, task=task, root=bids_root_path)
        raw = mne.io.read_raw_edf(os.path.join(edf_path, sub, eeg_path, edf_file), preload=False)
        raw.info["line_freq"] = 50
        write_raw_bids(raw, bids_path, overwrite=True)